In [2]:
pip install cftime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.5 MB/s eta 0:00:00


In [3]:
import os
import requests
import xarray as xr
from tqdm import tqdm
import pandas as pd
import cftime

BASE_URL = "https://www.ncei.noaa.gov/data/oceans/woa/WOA23/DATA/"

FILES = {
    "Temperature": "temperature/netcdf/decav91C0/1.00/woa23_decav91C0_t{month}_01.nc",
    "Salinity": "salinity/netcdf/decav91C0/1.00/woa23_decav91C0_s{month}_01.nc",
    "Oxygen": "oxygen/netcdf/all/1.00/woa23_all_o{month}_01.nc",
    "Nitrate": "nitrate/netcdf/all/1.00/woa23_all_n{month}_01.nc",
    "Phosphate": "phosphate/netcdf/all/1.00/woa23_all_p{month}_01.nc"
}

def download_file(url, filename):
    response = requests.get(url, stream=True)
    response.raise_for_status()
    total_size = int(response.headers.get('content-length', 0))
    with open(filename, 'wb') as f, tqdm(
        desc=os.path.basename(filename),
        total=total_size,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
    ) as pbar:
        for chunk in response.iter_content(8192):
            f.write(chunk)
            pbar.update(len(chunk))

def decode_time(ds):
    time_var = ds.time
    units = time_var.attrs.get('units', '')
    calendar = time_var.attrs.get('calendar', 'standard')

    if units.startswith("months since"):
        calendar = '360_day'

    times = cftime.num2date(time_var.values, units=units, calendar=calendar)
    # Konwertujemy do pandas datetime (na 1-szy dzień miesiąca)
    times_pd = pd.to_datetime([f"{t.year}-{t.month:02d}-01" for t in times])
    ds['time'] = ('time', times_pd)
    return ds


os.makedirs("WOA23_monthly", exist_ok=True)

for month in range(1, 13):
    month_str = f"{month:02d}"
    print(f"\n=== Month {month_str} ===")

    for var, template in FILES.items():
        filepath = template.format(month=month_str)
        url = BASE_URL + filepath
        filename = os.path.join("WOA23_monthly", os.path.basename(filepath))

        if not os.path.exists(filename):
            print(f"Downloading {var}...")
            download_file(url, filename)

        ds = xr.open_dataset(filename, decode_times=False)
        ds = decode_time(ds)

        var_name = {
            "Temperature": "t_an",
            "Salinity": "s_an",
            "Oxygen": "o_an",
            "Nitrate": "n_an",
            "Phosphate": "p_an"
        }[var]

        print(f"\n{var} (Month {month_str}):")
        print("Variables:", list(ds.variables.keys()))

        df = ds[var_name].to_dataframe().dropna()
        print(f"Data without NaNs ({len(df)} rows):")
        print(df)

print("\nAll monthly data processed!")



=== Month 01 ===


woa23_decav91C0_t01_01.nc: 100%|██████████| 57.0M/57.0M [00:38<00:00, 1.55MB/s]



Temperature (Month 01):
Variables: ['crs', 'lat', 'lat_bnds', 'lon', 'lon_bnds', 'depth', 'depth_bnds', 'time', 'climatology_bounds', 't_an', 't_mn', 't_dd', 't_sd', 't_se', 't_oa', 't_ma', 't_gp', 't_sdo', 't_sea']
Data without NaNs (2183675 rows):
                                   t_an
time       depth  lat   lon            
2005-01-01 0.0    -77.5 -178.5 -0.32070
                        -177.5 -0.36500
                        -176.5 -0.40521
                        -175.5 -0.42581
                        -174.5 -0.44261
...                                 ...
           1500.0  89.5  175.5 -0.48721
                         176.5 -0.48721
                         177.5 -0.48721
                         178.5 -0.48721
                         179.5 -0.48721

[2183675 rows x 1 columns]


woa23_decav91C0_s01_01.nc: 100%|██████████| 50.8M/50.8M [00:00<00:00, 99.4MB/s]



Salinity (Month 01):
Variables: ['crs', 'lat', 'lat_bnds', 'lon', 'lon_bnds', 'depth', 'depth_bnds', 'time', 'climatology_bounds', 's_an', 's_mn', 's_dd', 's_sd', 's_se', 's_oa', 's_ma', 's_gp', 's_sdo', 's_sea']
Data without NaNs (2183675 rows):
                                     s_an
time       depth  lat   lon              
2005-01-01 0.0    -77.5 -178.5  34.200310
                        -177.5  34.186913
                        -176.5  34.175400
                        -175.5  34.168610
                        -174.5  34.157711
...                                   ...
           1500.0  89.5  175.5  34.925499
                         176.5  34.925499
                         177.5  34.925499
                         178.5  34.925499
                         179.5  34.925499

[2183675 rows x 1 columns]


...


Oxygen (Month 12):
Variables: ['crs', 'lat', 'lat_bnds', 'lon', 'lon_bnds', 'depth', 'depth_bnds', 'time', 'climatology_bounds', 'o_an', 'o_mn', 'o_dd', 'o_sd', 'o_se', 'o_oa', 'o_ma', 'o_gp', 'o_sea']
Data without NaNs (2183675 rows):
                                      o_an
time       depth  lat   lon               
1993-12-01 0.0    -77.5 -178.5  380.080170
                        -177.5  379.472443
                        -176.5  378.745209
                        -175.5  378.228729
                        -174.5  377.659607
...                                    ...
           1500.0  89.5  175.5  297.694733
                         176.5  297.694733
                         177.5  297.694733
                         178.5  297.694733
                         179.5  297.694733

[2183675 rows x 1 columns]


woa23_all_n12_01.nc: 100%|██████████| 18.2M/18.2M [00:00<00:00, 55.9MB/s]



Nitrate (Month 12):
Variables: ['crs', 'lat', 'lat_bnds', 'lon', 'lon_bnds', 'depth', 'depth_bnds', 'time', 'climatology_bounds', 'n_an', 'n_mn', 'n_dd', 'n_sd', 'n_se', 'n_oa', 'n_ma', 'n_gp', 'n_sea']
Data without NaNs (1680367 rows):
                                    n_an
time       depth lat   lon              
1993-12-01 0.0   -77.5 -178.5  14.865408
                       -177.5  14.976521
                       -176.5  15.049278
                       -175.5  15.053966
                       -174.5  14.934766
...                                  ...
           800.0  89.5  175.5  12.129090
                        176.5  12.129090
                        177.5  12.129090
                        178.5  12.129090
                        179.5  12.129090

[1680367 rows x 1 columns]


woa23_all_p12_01.nc: 100%|██████████| 20.0M/20.0M [00:00<00:00, 76.8MB/s]



Phosphate (Month 12):
Variables: ['crs', 'lat', 'lat_bnds', 'lon', 'lon_bnds', 'depth', 'depth_bnds', 'time', 'climatology_bounds', 'p_an', 'p_mn', 'p_dd', 'p_sd', 'p_se', 'p_oa', 'p_ma', 'p_gp', 'p_sea']
Data without NaNs (1680367 rows):
                                   p_an
time       depth lat   lon             
1993-12-01 0.0   -77.5 -178.5  1.345868
                       -177.5  1.342871
                       -176.5  1.341775
                       -175.5  1.343025
                       -174.5  1.346306
...                                 ...
           800.0  89.5  175.5  0.805623
                        176.5  0.805623
                        177.5  0.805623
                        178.5  0.805623
                        179.5  0.805623

[1680367 rows x 1 columns]

All monthly data processed!
